# Capstone - Week 4 - part 2

## importing the requered librarys

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


## the data for paris borough, Neighborhood, coordonates from the official web site of the Paris town :
https://opendata.paris.fr/explore/dataset/quartier_paris/information/

In [3]:
df=pd.read_csv("quartier_paris.csv",sep=";")

In [4]:
df.head()

,N_SQ_QU,C_QU,C_QUINSEE,L_QU,C_AR,N_SQ_AR,PERIMETRE,SURFACE,Geometry X Y,Geometry
0,750000021,21,7510601,Monnaie,6,750000006,2391.122818,2.933606e+05,"48.8543844036,2.34003537113","{""type"": ""Polygon"", ""coordinates"": [[[2.343168..."
1,750000022,22,7510602,Odéon,6,750000006,3516.314464,7.161484e+05,"48.8478006293,2.33633882759","{""type"": ""Polygon"", ""coordinates"": [[[2.336988..."
2,750000029,29,7510801,Champs-Elysées,8,750000008,5480.652870,1.140254e+06,"48.8670744922,2.30865168468","{""type"": ""Polygon"", ""coordinates"": [[[2.306463..."
3,750000051,51,7511303,Maison-Blanche,13,750000013,6814.373288,2.231506e+06,"48.8231278057,2.35243314954","{""type"": ""Polygon"", ""coordinates"": [[[2.349115..."
4,750000052,52,7511304,Croulebarbe,13,750000013,3289.230480,6.920677e+05,"48.8337336761,2.34767304607","{""type"": ""Polygon"", ""coordinates"": [[[2.351662..."


## selecting the data that interest us:
 - **Geometry X Y**	: contain latitude and longitude of Neighborhood, we need to split this column into tow (longitude of latitude)
 - **L_QU** : contain the name of the Neighborhood
 - **C_AR** : containe the borough of each Neighborhood , you can check on the wiki page to know more about Paris :
 https://en.wikipedia.org/wiki/Quarters_of_Paris

In [5]:
df1=df[['L_QU','C_AR']]

In [6]:
df1['latitude']=df['Geometry X Y'].apply(lambda x : x.split(',')[0])
df1['longitude']=df['Geometry X Y'].apply(lambda x : x.split(',')[1])

<ipython-input-6-0dda3f2cf8d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['latitude']=df['Geometry X Y'].apply(lambda x : x.split(',')[0])
<ipython-input-6-0dda3f2cf8d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['longitude']=df['Geometry X Y'].apply(lambda x : x.split(',')[1])


## Paris coordonates using geolocator

In [7]:
address = 'Paris'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


## Making latitude and longitude float type

In [8]:
df1['latitude']=df1['latitude'].astype('float64')
df1['longitude']=df1['longitude'].astype('float64')
df1

<ipython-input-8-4e6c14395472>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['latitude']=df1['latitude'].astype('float64')
<ipython-input-8-4e6c14395472>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['longitude']=df1['longitude'].astype('float64')


,L_QU,C_AR,latitude,longitude
0,Monnaie,6,48.854384,2.340035
1,Odéon,6,48.847801,2.336339
2,Champs-Elysées,8,48.867074,2.308652
3,Maison-Blanche,13,48.823128,2.352433
4,Croulebarbe,13,48.833734,2.347673
...,...,...,...,...
75,Clignancourt,18,48.891668,2.345979
76,Goutte-d'Or,18,48.892138,2.355536
77,Rochechouart,9,48.879812,2.344861
78,Bercy,12,48.835209,2.386210


# Neighborhood of paris on map

In [9]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['latitude'], df1['longitude'], df1['C_AR'], df1['L_QU']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

# Getting 30 first venues of each neighberhood on data frame
## foursquare credentials

In [10]:
CLIENT_ID = '54KLDE4RKR1GUX2NM1CIRTXXZVQOEEHRBIPZ2WSI4JDJFRUB' # your Foursquare ID
CLIENT_SECRET = 'BHQIWNOEBWZQ2VLHADSC0YIKUUEHUO5YS4BFAKFP41URSI04' # your Foursquare Secret
ACCESS_TOKEN = 'YJSOHJD14FQFTQ0HDCSBPAS2SAUX3Z031NCH5YFDJCSWPN2K' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 30 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 54KLDE4RKR1GUX2NM1CIRTXXZVQOEEHRBIPZ2WSI4JDJFRUB
CLIENT_SECRET:BHQIWNOEBWZQ2VLHADSC0YIKUUEHUO5YS4BFAKFP41URSI04


# getNearbyVenues function : make serach venues on each Neighborhood

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
paris_venues = getNearbyVenues(names=df1['L_QU'],
                                   latitudes=df1['latitude'],
                                   longitudes=df1['longitude']
                                  )

Monnaie
Odéon
Champs-Elysées
Maison-Blanche
Croulebarbe
Sainte-Avoie
Hôpital-Saint-Louis
Belleville
Ternes
Folie-Méricourt
Salpêtrière
Place-Vendôme
Combat
Charonne
Javel
Ecole-Militaire
Grenelle
Auteuil
Quinze-Vingts
Saint-Gervais
Saint-Thomas-d'Aquin
Porte-Saint-Denis
Saint-Germain-l'Auxerrois
Villette
Arts-et-Métiers
Archives
Faubourg-du-Roule
Arsenal
Jardin-des-Plantes
Porte-Saint-Martin
Roquette
Picpus
Plaisance
Invalides
Faubourg-Montmartre
Gaillon
Amérique
Saint-Merri
Notre-Dame
Gros-Caillou
Vivienne
Enfants-Rouges
Saint-Germain-des-Prés
Saint-Vincent-de-Paul
Saint-Ambroise
Bel-Air
Montparnasse
Plaine de Monceaux
Europe
Sainte-Marguerite
Parc-de-Montsouris
Saint-Lambert
Val-de-Grâce
Necker
Père-Lachaise
La Chapelle
Saint-Victor
Madeleine
Saint-Fargeau
Porte-Dauphine
Grandes-Carrières
Batignolles
Notre-Dame-des-Champs
Petit-Montrouge
Pont-de-Flandre
Muette
Chaillot
Epinettes
Sorbonne
Saint-Georges
Chaussée-d'Antin
Palais-Royal
Mail
Bonne-Nouvelle
Gare
Clignancourt
Goutte-d'Or
Roc

## Our final dataframe contain the following data:

- paris Neighborhood
- Neighborhood Latitude	
- Neighborhood Longitude	
- Venue
- Venue Latitude	
- Venue Longitude	
- Venue Category


In [15]:
paris_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Monnaie,48.854384,2.340035,Chez Nous,48.855823,2.340271,Wine Bar
1,Monnaie,48.854384,2.340035,Chez Le Libanais,48.853285,2.341673,Lebanese Restaurant
2,Monnaie,48.854384,2.340035,Maison Sauvage,48.853744,2.337993,French Restaurant
3,Monnaie,48.854384,2.340035,Taschen Store,48.853750,2.338162,Bookstore
4,Monnaie,48.854384,2.340035,Ze Kitchen Galerie,48.855101,2.341297,Restaurant
...,...,...,...,...,...,...,...
2203,Halles,48.862289,2.344899,Église Saint-Germain-l'Auxerrois (Église Saint...,48.859520,2.341306,Church
2204,Halles,48.862289,2.344899,Stohrer,48.865253,2.346816,Pastry Shop
2205,Halles,48.862289,2.344899,LECLAIREUR Hérold,48.864832,2.341530,Clothing Store
2206,Halles,48.862289,2.344899,Tir-Bouchon,48.864648,2.348728,French Restaurant
